In [1]:
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.5,contrast=0.5,saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.255])
])

In [3]:
class Dataset(Dataset):
    def __init__(self,image_paths,transform=None):
        self.image_paths=image_paths
        self.transform=transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self,idx):
        image_path=self.image_paths[idx]
        image = Image.open(image_path)
        if self.transform:
            image=self.transform(image)
        label = 1
        return image, label

In [4]:
celeb_path = r"C:\Users\oppor\Desktop\RajeshHamal"
celebrity_images = [os.path.join(celeb_path, filename) for filename in os.listdir(celeb_path) if filename.endswith(".jpg")]

In [5]:
rajeshdai = Dataset(image_paths=celebrity_images,transform=transform)
data_loader=DataLoader(rajeshdai,batch_size=32,shuffle=True)

In [6]:
import torch.nn as nn
import torchvision.models as models

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)  # Binary output (0 or 1)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

C:\Users\oppor\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\oppor\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\oppor/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [04:50<00:00, 161kB/s]


In [14]:
# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for images, labels in data_loader:
        # Move data to device
        images, labels = images.to(device), labels.to(device).float()
        
        # Forward pass
        outputs = model(images)
        
        # Calculate loss
        loss = criterion(outputs.squeeze(), labels)  # Squeeze to remove the extra dimension
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(data_loader)}")


Epoch 1/5, Loss: 0.6639777421951294
Epoch 2/5, Loss: 0.69481360912323
Epoch 3/5, Loss: 0.28432077169418335
Epoch 4/5, Loss: 0.16535209119319916
Epoch 5/5, Loss: 0.09148901700973511


In [43]:
# Prediction function
def predict(image_path, model):
    model.eval()  # Set the model to evaluation mode
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)  # Apply the transformation and add batch dimension
    
    with torch.no_grad():
        output = model(image)  # Get model output (probability score)
        output = output.item()  # Convert to Python scalar
    
    # Output: 1 if the model recognizes the celebrity, 0 if not
    if output > 0.5:
        return "This is the rajeshhamal!"
    else:
        return "This is not the celebrity."

# Test prediction with a new image
print(predict(r"C:\Users\oppor\Desktop\download.jpg", model))


This is not the celebrity.


In [45]:
torch.save(model.state_dict(),'rajesh_hamal_model.pth')